# 🎯 **Use Case: Predicting Attraction Ratings**

> *Aim:*
- Develop a regression model to predict the rating a user might give to a tourist attraction based on historical data, user demographics, and attraction features.

> *Use Case:*
- Travel platforms can use this model to estimate the satisfaction level of users visiting specific attractions. 
- By identifying attractions likely to receive lower ratings, agencies can take corrective actions, such as improving services or better setting user expectations.
- Personal travel guides can provide users with attractions most aligned with their preferences to enhance overall satisfaction.

> *Possible Inputs (Features):*
- User demographics: Continent, region, country, city.
- Visit details: Year, month, mode of visit (e.g., business, family, friends).
- Attraction attributes: Type (e.g., beaches, ruins), location, and previous average ratings.

> *Target:*
- Predicted rating (on a scale, e.g., 1-5).


In [72]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [75]:
pip install lightgbm


   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ------- -------------------------------- 0.3/1.5 MB ? eta -:--:--
   -------------- ------------------------- 0.5/1.5 MB 932.9 kB/s eta 0:00:01
   -------------- ------------------------- 0.5/1.5 MB 932.9 kB/s eta 0:00:01
   --------------------- ------------------ 0.8/1.5 MB 745.8 kB/s eta 0:00:01
   --------------------- ------------------ 0.8/1.5 MB 745.8 kB/s eta 0:00:01
   ---------------------------- ----------- 1.0/1.5 MB 699.0 kB/s eta 0:00:01
   ---------------------------- ----------- 1.0/1.5 MB 699.0 kB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 718.2 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [7]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder

import pickle

# Tree-Based Regressors
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

# XGBoost Regressor
from xgboost import XGBRegressor

# LightGBM Regressor
from lightgbm import LGBMRegressor

# Preprocessing & Model Selection, Model Evaluation metrics(MAE, MSE, r2_Score)
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [15]:
# Load preprocessed dataset
Tourism_data = pd.read_csv("C:/Users/dhars/Downloads/Dhass/codeing/GUVI/2. MainBoot/4.Project_Code/Project4/Project4_Code/Tourism_Experince_Analytics/DataSets/PreProcessed_DataSet/Pre_Processed_DataSet.csv")

In [16]:
Tourism_data.head(5)

,UserId,TransactionId,VisitYear,VisitMonth,Rating,VisitDate,VisitMode,Attraction,AttractionAddress,AttractionType,CityName,Country,Region,Continent,Average rating per user,Total visits by VisitMode per user,Average rating per attraction,Total visits per user
0,70456,3,2022,10,1.0,2022-10-01,Couples,Sacred Monkey Forest Sanctuary,"Jl. Monkey Forest, Ubud 80571 Indonesia",Nature & Wildlife Areas,Kerala,India,South East Asia,Asia,1.0,1,0.663674,1
1,7567,8,2022,10,1.0,2022-10-01,Friends,Sacred Monkey Forest Sanctuary,"Jl. Monkey Forest, Ubud 80571 Indonesia",Nature & Wildlife Areas,Belleville,Canada,South America,America,1.0,1,0.663674,1
2,79069,9,2022,10,1.0,2022-10-01,Family,Sacred Monkey Forest Sanctuary,"Jl. Monkey Forest, Ubud 80571 Indonesia",Nature & Wildlife Areas,Squamish,Canada,South America,America,1.0,1,0.663674,1
3,31019,10,2022,10,0.5,2022-10-01,Family,Sacred Monkey Forest Sanctuary,"Jl. Monkey Forest, Ubud 80571 Indonesia",Nature & Wildlife Areas,Innisfil,Canada,South America,America,0.5,2,0.663674,2
4,43611,15,2022,10,0.5,2022-10-01,Couples,Sacred Monkey Forest Sanctuary,"Jl. Monkey Forest, Ubud 80571 Indonesia",Nature & Wildlife Areas,Crown Point,United States,South America,America,0.5,3,0.663674,3


In [17]:
#Drop Unnecessary Columns
Tourism_data.drop(['AttractionAddress','TransactionId','VisitDate','Total visits by VisitMode per user','Total visits per user'], axis= 1, inplace= True)

In [18]:
Tourism_data.head(2)

,UserId,VisitYear,VisitMonth,Rating,VisitMode,Attraction,AttractionType,CityName,Country,Region,Continent,Average rating per user,Average rating per attraction
0,70456,2022,10,1.0,Couples,Sacred Monkey Forest Sanctuary,Nature & Wildlife Areas,Kerala,India,South East Asia,Asia,1.0,0.663674
1,7567,2022,10,1.0,Friends,Sacred Monkey Forest Sanctuary,Nature & Wildlife Areas,Belleville,Canada,South America,America,1.0,0.663674


In [19]:
# Specify columns to encode
encoding_columns = Tourism_data[['VisitMode', 'Attraction', 'AttractionType', 'CityName','Country', 'Region', 'Continent']]
encoding_columns.head()

,VisitMode,Attraction,AttractionType,CityName,Country,Region,Continent
0,Couples,Sacred Monkey Forest Sanctuary,Nature & Wildlife Areas,Kerala,India,South East Asia,Asia
1,Friends,Sacred Monkey Forest Sanctuary,Nature & Wildlife Areas,Belleville,Canada,South America,America
2,Family,Sacred Monkey Forest Sanctuary,Nature & Wildlife Areas,Squamish,Canada,South America,America
3,Family,Sacred Monkey Forest Sanctuary,Nature & Wildlife Areas,Innisfil,Canada,South America,America
4,Couples,Sacred Monkey Forest Sanctuary,Nature & Wildlife Areas,Crown Point,United States,South America,America


In [20]:
encoding_columns.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62223 entries, 0 to 62222
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   VisitMode       62223 non-null  object
 1   Attraction      62223 non-null  object
 2   AttractionType  62223 non-null  object
 3   CityName        62223 non-null  object
 4   Country         62223 non-null  object
 5   Region          62223 non-null  object
 6   Continent       62223 non-null  object
dtypes: object(7)
memory usage: 3.3+ MB


In [21]:
encoders = {}

for column in encoding_columns:
    le = LabelEncoder()
    Tourism_data[column] = le.fit_transform(Tourism_data[column]) #Fit and encode the column
    encoders[column] = le #Store encoder for future use

In [22]:
encoders

{'VisitMode': LabelEncoder(),
 'Attraction': LabelEncoder(),
 'AttractionType': LabelEncoder(),
 'CityName': LabelEncoder(),
 'Country': LabelEncoder(),
 'Region': LabelEncoder(),
 'Continent': LabelEncoder()}

- **Saving the encoder.pkl file:**

In [23]:
import pickle
with open ("C:/Users/dhars/Downloads/Dhass/codeing/GUVI/2. MainBoot/4.Project_Code/Project4/Project4_Code/Tourism_Experince_Analytics/models/encoder.pkl", "wb") as f:
    pickle.dump(encoders,f)

In [24]:
Tourism_data.head(2)

,UserId,VisitYear,VisitMonth,Rating,VisitMode,Attraction,AttractionType,CityName,Country,Region,Continent,Average rating per user,Average rating per attraction
0,70456,2022,10,1.0,1,17,8,2266,46,14,2,1.0,0.663674
1,7567,2022,10,1.0,3,17,8,425,21,12,1,1.0,0.663674


- ***Define features and target***

In [25]:
#Define features and target:
x = Tourism_data.drop(columns=['Rating'])
y = Tourism_data['Rating']

In [26]:
x.head(2)

,UserId,VisitYear,VisitMonth,VisitMode,Attraction,AttractionType,CityName,Country,Region,Continent,Average rating per user,Average rating per attraction
0,70456,2022,10,1,17,8,2266,46,14,2,1.0,0.663674
1,7567,2022,10,3,17,8,425,21,12,1,1.0,0.663674


In [27]:
y.head(2)

0    1.0
1    1.0
Name: Rating, dtype: float64

In [28]:
x.shape, y.shape

((62223, 12), (62223,))

- ***Model Training: Train/test split***

In [29]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size= 0.2, random_state= 5)

In [30]:
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((49778, 12), (12445, 12), (49778,), (12445,))

In [31]:
x_train

,UserId,VisitYear,VisitMonth,VisitMode,Attraction,AttractionType,CityName,Country,Region,Continent,Average rating per user,Average rating per attraction
18132,10647,2017,1,2,28,15,3951,77,10,1,0.928571,0.959338
6247,49307,2017,2,1,17,8,654,108,12,1,0.750000,0.663674
26650,59122,2018,7,1,14,2,4533,108,12,1,1.000000,0.670358
26496,35481,2019,1,2,14,2,4981,108,12,1,1.000000,0.670358
23646,29600,2018,2,1,19,2,1222,108,12,1,0.625000,0.300270
...,...,...,...,...,...,...,...,...,...,...,...,...
35814,87907,2014,3,2,26,11,1921,26,8,2,0.750000,0.626533
59069,56404,2017,6,2,16,0,547,108,12,1,0.750000,0.405954
20463,76725,2016,1,2,28,15,4506,49,13,2,0.800000,0.959338
18638,38279,2016,9,3,28,15,3950,108,12,1,1.000000,0.959338


In [32]:
y_train

18132    1.00
6247     0.75
26650    1.00
26496    1.00
23646    0.50
         ... 
35814    0.75
59069    0.75
20463    0.75
18638    1.00
35683    0.75
Name: Rating, Length: 49778, dtype: float64

In [33]:
x_test

,UserId,VisitYear,VisitMonth,VisitMode,Attraction,AttractionType,CityName,Country,Region,Continent,Average rating per user,Average rating per attraction
15779,85715,2019,10,2,28,15,5379,46,14,2,1.000000,0.959338
45172,16975,2016,3,1,24,16,1366,21,12,1,0.875000,0.374137
41810,62812,2015,10,1,23,10,2435,56,13,2,1.000000,0.578319
9983,79590,2015,11,3,17,8,485,108,12,1,0.666667,0.663674
58665,47370,2014,6,4,15,1,1541,21,12,1,1.000000,0.596119
...,...,...,...,...,...,...,...,...,...,...,...,...
29061,61864,2018,8,2,18,2,3370,7,11,3,1.000000,0.437173
51532,56377,2016,10,4,8,2,1366,21,12,1,1.000000,0.000000
22766,6371,2020,3,1,19,2,3951,77,10,1,0.750000,0.300270
60278,50878,2018,4,4,25,6,547,108,12,1,0.875000,0.655541


In [34]:
default_country = sorted(encoders['Country'].classes_)[46]
default_country

'India'

In [35]:
y_test

15779    1.00
45172    0.75
41810    1.00
9983     0.75
58665    1.00
         ... 
29061    1.00
51532    1.00
22766    0.25
60278    1.00
50401    0.75
Name: Rating, Length: 12445, dtype: float64

- Defining Models:

In [36]:
gb_regressor = GradientBoostingRegressor()
decision_tree_regressor = DecisionTreeRegressor()
randomForest_regressor = RandomForestRegressor()
xgb_regressor = XGBRegressor()
lgb_regressor = LGBMRegressor()

- Hyperparameter tuning using RandomizedSearchCV

In [37]:
# 1. GradientBoostingRegressor:
gb_params = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'subsample': [0.6, 0.8, 1.0]
}

random_search_gb = RandomizedSearchCV(
    estimator=gb_regressor,
    param_distributions=gb_params,n_iter=10,cv=5,verbose=1,random_state=42,n_jobs=-1)

# 2. DecisionTreeRegressor
decision_params = {
    'max_depth': [5, 10, 15, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

random_search_decision = RandomizedSearchCV(
    estimator=decision_tree_regressor,
    param_distributions=decision_params,n_iter=10,cv=5,verbose=1,random_state=42,n_jobs=-1)

# 3. RandomForestRegressor
rf_params = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

random_search_rf = RandomizedSearchCV(
    estimator=randomForest_regressor,param_distributions=rf_params,n_iter=10,cv=5,verbose=1,random_state=42,n_jobs=-1)

#4. XGBRegressor:
xgb_params = {
    'n_estimators': [100, 200],
    'max_depth': [3, 6, 9],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0]
}

random_search_xgb = RandomizedSearchCV(
    estimator=xgb_regressor,
    param_distributions=xgb_params,n_iter=10,cv=5,verbose=1,random_state=42,n_jobs=-1)


#5. LGBMRegressor
lgb_params = {
    'n_estimators': [100, 200],
    'max_depth': [3, 6, 9, -1],
    'learning_rate': [0.01, 0.1, 0.2],
    'num_leaves': [31, 50, 100],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0]
}

random_search_lgb = RandomizedSearchCV(
    estimator=lgb_regressor,
    param_distributions=lgb_params,n_iter=10,cv=5,verbose=1,random_state=42,n_jobs=-1)


- Fit the RandomizedSearchCV Models

In [38]:
random_search_gb.fit(x_train, y_train)
random_search_decision.fit(x_train, y_train)
random_search_rf.fit(x_train, y_train)
random_search_xgb.fit(x_train, y_train)
random_search_lgb.fit(x_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Fitting 5 folds for each of 10 candidates, totalling 50 fits
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003182 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 850
[LightGBM] [Info] Number of data points in the train set: 49778, number of used features: 12
[LightGBM] [Info] Start training from score 0.789450


,estimator,LGBMRegressor()
,param_distributions,"{'colsample_bytree': [0.6, 0.8, ...], 'learning_rate': [0.01, 0.1, ...], 'max_depth': [3, 6, ...], 'n_estimators': [100, 200], ...}"
,n_iter,10
,scoring,None
,n_jobs,-1
,refit,True
,cv,5
,verbose,1
,pre_dispatch,'2*n_jobs'
,random_state,42
,error_score,nan


- Best Parametrs of the Models:

In [39]:
print("Best GBM Params:", random_search_gb.best_params_)
print("Best Decision Tree Params:", random_search_decision.best_params_)
print("Best Random Forest Params:", random_search_rf.best_params_)
print("Best XGBoost Params:", random_search_xgb.best_params_)
print("Best LightGBM Params:", random_search_lgb.best_params_)

Best GBM Params: {'subsample': 0.8, 'n_estimators': 300, 'max_depth': 5, 'learning_rate': 0.2}
Best Decision Tree Params: {'min_samples_split': 10, 'min_samples_leaf': 4, 'max_depth': 5}
Best Random Forest Params: {'n_estimators': 200, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': 20}
Best XGBoost Params: {'subsample': 1.0, 'n_estimators': 100, 'max_depth': 9, 'learning_rate': 0.2, 'colsample_bytree': 1.0}
Best LightGBM Params: {'subsample': 0.6, 'num_leaves': 100, 'n_estimators': 200, 'max_depth': -1, 'learning_rate': 0.2, 'colsample_bytree': 0.8}


- Best estimator of the Models:

In [40]:
random_search_rf.best_estimator_ #It will show the best parameters of the model

,n_estimators,200
,criterion,'squared_error'
,max_depth,20
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,1.0
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [41]:
random_search_xgb.best_estimator_

,objective,'reg:squarederror'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,1.0
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [42]:
random_search_lgb.best_estimator_

,boosting_type,'gbdt'
,num_leaves,100
,max_depth,-1
,learning_rate,0.2
,n_estimators,200
,subsample_for_bin,200000
,objective,None
,class_weight,None
,min_split_gain,0.0
,min_child_weight,0.001
,min_child_samples,20


In [43]:
random_search_gb.best_estimator_

,loss,'squared_error'
,learning_rate,0.2
,n_estimators,300
,subsample,0.8
,criterion,'friedman_mse'
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_depth,5
,min_impurity_decrease,0.0
,init,None


In [44]:
random_search_decision.best_estimator_

,criterion,'squared_error'
,splitter,'best'
,max_depth,5
,min_samples_split,10
,min_samples_leaf,4
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,None
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,ccp_alpha,0.0


- Evaluation Metrics of Each Model:

In [45]:
def evaluate(model, name="Model"):
    y_pred = model.predict(x_test)
    print(f"\n📊 {name} Performance:")
    print(f"MAE  : {mean_absolute_error(y_test, y_pred):.4f}")
    print(f"MSE  : {mean_squared_error(y_test, y_pred):.4f}")
    print(f"RMSE : {np.sqrt(mean_squared_error(y_test, y_pred)):.4f}")
    print(f"R²   : {r2_score(y_test, y_pred):.4f}")

# Evaluate all
evaluate(random_search_rf.best_estimator_, "Random Forest")
evaluate(random_search_xgb.best_estimator_, "XGBoost")
evaluate(random_search_lgb.best_estimator_, "LightGBM")
evaluate(random_search_gb.best_estimator_, "GradientBoosting")
evaluate(random_search_decision.best_estimator_, "Decision Tree")


📊 Random Forest Performance:
MAE  : 0.0660
MSE  : 0.0146
RMSE : 0.1207
R²   : 0.7451

📊 XGBoost Performance:
MAE  : 0.0751
MSE  : 0.0160
RMSE : 0.1264
R²   : 0.7203

📊 LightGBM Performance:
MAE  : 0.0801
MSE  : 0.0167
RMSE : 0.1294
R²   : 0.7066

📊 GradientBoosting Performance:
MAE  : 0.0828
MSE  : 0.0180
RMSE : 0.1341
R²   : 0.6852

📊 Decision Tree Performance:
MAE  : 0.0800
MSE  : 0.0190
RMSE : 0.1379
R²   : 0.6667


In [46]:
# Get predictions
y_pred_decision = random_search_decision.best_estimator_.predict(x_test)
y_pred_rf = random_search_rf.best_estimator_.predict(x_test)
y_pred_xgb = random_search_xgb.best_estimator_.predict(x_test)
y_pred_lgb = random_search_lgb.best_estimator_.predict(x_test)
y_pred_gb = random_search_gb.best_estimator_.predict(x_test)

# Store results
results = {
    'Model': ['Decision Tree', 'Random Forest', 'XGBoost', 'LightGBM', 'GradientBoosting'],
    'R2 Score': [
        r2_score(y_test, y_pred_decision),
        r2_score(y_test, y_pred_rf),
        r2_score(y_test, y_pred_xgb),
        r2_score(y_test, y_pred_lgb),
        r2_score(y_test, y_pred_gb)
    ],
    'MAE': [
        mean_absolute_error(y_test, y_pred_decision),
        mean_absolute_error(y_test, y_pred_rf),
        mean_absolute_error(y_test, y_pred_xgb),
        mean_absolute_error(y_test, y_pred_lgb),
        mean_absolute_error(y_test, y_pred_gb)
    ],
    'MSE': [
        mean_squared_error(y_test, y_pred_decision),
        mean_squared_error(y_test, y_pred_rf),
        mean_squared_error(y_test, y_pred_xgb),
        mean_squared_error(y_test, y_pred_lgb),
        mean_squared_error(y_test, y_pred_gb)
    ]
}

# Convert to DataFrame
results_df = pd.DataFrame(results)
results_df

,Model,R2 Score,MAE,MSE
0,Decision Tree,0.666705,0.080036,0.019030
1,Random Forest,0.745054,0.066034,0.014557
2,XGBoost,0.720290,0.075084,0.015970
3,LightGBM,0.706639,0.080125,0.016750
4,GradientBoosting,0.685217,0.082795,0.017973


- Save the Best Models

In [47]:
# Save each model to a file
with open("C:/Users/dhars/Downloads/Dhass/codeing/GUVI/2. MainBoot/4.Project_Code/Project4/Project4_Code/Tourism_Experince_Analytics/models/Regression_Task_Model/best_rfr_model.pkl", "wb") as f:
    pickle.dump(random_search_rf.best_estimator_, f)

with open("C:/Users/dhars/Downloads/Dhass/codeing/GUVI/2. MainBoot/4.Project_Code/Project4/Project4_Code/Tourism_Experince_Analytics/models/Regression_Task_Model/best_xgb_model.pkl", "wb") as f:
    pickle.dump(random_search_xgb.best_estimator_, f)

with open("C:/Users/dhars/Downloads/Dhass/codeing/GUVI/2. MainBoot/4.Project_Code/Project4/Project4_Code/Tourism_Experince_Analytics/models/Regression_Task_Model/best_lgb_model.pkl", "wb") as f:
    pickle.dump(random_search_lgb.best_estimator_, f)

with open("C:/Users/dhars/Downloads/Dhass/codeing/GUVI/2. MainBoot/4.Project_Code/Project4/Project4_Code/Tourism_Experince_Analytics/models/Regression_Task_Model/best_gb_model.pkl", "wb") as f:
    pickle.dump(random_search_gb.best_estimator_, f)

with open("C:/Users/dhars/Downloads/Dhass/codeing/GUVI/2. MainBoot/4.Project_Code/Project4/Project4_Code/Tourism_Experince_Analytics/models/Regression_Task_Model/best_decision_tree_model.pkl", "wb") as f:
    pickle.dump(random_search_decision.best_estimator_, f)


- Saving the pkl file in joblib to deploy in cloud purpose:

In [48]:
from joblib import dump
from joblib import load
import pickle

In [49]:
# Load encoders
with open('C:/Users/dhars/Downloads/Dhass/codeing/GUVI/2. MainBoot/4.Project_Code/Project4/Project4_Code/Tourism_Experince_Analytics/models/encoder.pkl', 'rb') as f:
    e = pickle.load(f)

# Save it using joblib with compression
dump(e, "C:/Users/dhars/Downloads/Dhass/codeing/GUVI/2. MainBoot/4.Project_Code/Project4/Project4_Code/Tourism_Experince_Analytics/models/encoder.joblib", compress=3)

encoders = load("C:/Users/dhars/Downloads/Dhass/codeing/GUVI/2. MainBoot/4.Project_Code/Project4/Project4_Code/Tourism_Experince_Analytics/models/encoder.joblib")

In [50]:
# Load random_Forest_regressor_model
with open("C:/Users/dhars/Downloads/Dhass/codeing/GUVI/2. MainBoot/4.Project_Code/Project4/Project4_Code/Tourism_Experince_Analytics/models/Regression_Task_Model/best_rfr_model.pkl", 'rb') as f:
    r = pickle.load(f)

# Save it using joblib with compression
dump(r, "C:/Users/dhars/Downloads/Dhass/codeing/GUVI/2. MainBoot/4.Project_Code/Project4/Project4_Code/Tourism_Experince_Analytics/models/Regression_Task_Model/best_rfr_model.joblib", compress=3)

rfr = load("C:/Users/dhars/Downloads/Dhass/codeing/GUVI/2. MainBoot/4.Project_Code/Project4/Project4_Code/Tourism_Experince_Analytics/models/Regression_Task_Model/best_rfr_model.joblib")

In [51]:
# Load Extreme_Gradient_Boosting_model
with open("C:/Users/dhars/Downloads/Dhass/codeing/GUVI/2. MainBoot/4.Project_Code/Project4/Project4_Code/Tourism_Experince_Analytics/models/Regression_Task_Model/best_xgb_model.pkl", 'rb') as f:
    r = pickle.load(f)

# Save it using joblib with compression
dump(r, "C:/Users/dhars/Downloads/Dhass/codeing/GUVI/2. MainBoot/4.Project_Code/Project4/Project4_Code/Tourism_Experince_Analytics/models/Regression_Task_Model/best_xgb_model.joblib", compress=3)

xgbm = load("C:/Users/dhars/Downloads/Dhass/codeing/GUVI/2. MainBoot/4.Project_Code/Project4/Project4_Code/Tourism_Experince_Analytics/models/Regression_Task_Model/best_xgb_model.joblib")

In [52]:
# Load Light_Gradient_Boosting_model
with open("C:/Users/dhars/Downloads/Dhass/codeing/GUVI/2. MainBoot/4.Project_Code/Project4/Project4_Code/Tourism_Experince_Analytics/models/Regression_Task_Model/best_lgb_model.pkl", 'rb') as f:
    r = pickle.load(f)

# Save it using joblib with compression
dump(r, "C:/Users/dhars/Downloads/Dhass/codeing/GUVI/2. MainBoot/4.Project_Code/Project4/Project4_Code/Tourism_Experince_Analytics/models/Regression_Task_Model/best_lgb_model.joblib", compress=3)

lgbm = load("C:/Users/dhars/Downloads/Dhass/codeing/GUVI/2. MainBoot/4.Project_Code/Project4/Project4_Code/Tourism_Experince_Analytics/models/Regression_Task_Model/best_lgb_model.joblib")

In [53]:
# Load Gradient_Boosting_model
with open("C:/Users/dhars/Downloads/Dhass/codeing/GUVI/2. MainBoot/4.Project_Code/Project4/Project4_Code/Tourism_Experince_Analytics/models/Regression_Task_Model/best_gb_model.pkl", 'rb') as f:
    r = pickle.load(f)

# Save it using joblib with compression
dump(r, "C:/Users/dhars/Downloads/Dhass/codeing/GUVI/2. MainBoot/4.Project_Code/Project4/Project4_Code/Tourism_Experince_Analytics/models/Regression_Task_Model/best_gb_model.joblib", compress=3)

gb = load("C:/Users/dhars/Downloads/Dhass/codeing/GUVI/2. MainBoot/4.Project_Code/Project4/Project4_Code/Tourism_Experince_Analytics/models/Regression_Task_Model/best_gb_model.joblib")

In [ ]:
# Load decision_tree_model
with open("C:/Users/dhars/Downloads/Dhass/codeing/GUVI/2. MainBoot/4.Project_Code/Project4/Project4_Code/Tourism_Experince_Analytics/models/Regression_Task_Model/best_decision_tree_model.pkl", 'rb') as f:
    r = pickle.load(f)

# Save it using joblib with compression
dump(r, "C:/Users/dhars/Downloads/Dhass/codeing/GUVI/2. MainBoot/4.Project_Code/Project4/Project4_Code/Tourism_Experince_Analytics/models/Regression_Task_Model/best_decision_tree_model.joblib", compress=3)

decision_tree = load("C:/Users/dhars/Downloads/Dhass/codeing/GUVI/2. MainBoot/4.Project_Code/Project4/Project4_Code/Tourism_Experince_Analytics/models/Regression_Task_Model/best_decision_tree_model.joblib")